In [22]:
import numpy as np
import pandas as pd
import xlrd
import xlwt
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import jupyterlab
import statsmodels.api as sm
from pandas_profiling import ProfileReport 
from feature_engine.creation import CyclicalTransformer
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
%matplotlib inline

In [23]:
training = pd.read_csv('training_data.csv', encoding = "latin")
testing = pd.read_csv('test_data.csv', encoding = "latin")
datasets = [training,testing]

In [24]:
for data in datasets:
 #data['AVERAGE_CLOUDINESS'] = data['AVERAGE_CLOUDINESS'].fillna(data['AVERAGE_CLOUDINESS'].mode()[0])
 data['AVERAGE_RAIN'] = data['AVERAGE_RAIN'].fillna('NoData')
 data['AVERAGE_CLOUDINESS'] = data['AVERAGE_CLOUDINESS'].fillna('NoData')

In [25]:
for data in datasets:

    data['record_date'] = pd.to_datetime(training['record_date'])

    data['weekday'] = data['record_date'].dt.dayofweek
    data['hour'] = data['record_date'].dt.hour
    data['month'] = data['record_date'].dt.month
    data['year'] = data['record_date'].dt.year
    data['day'] = data['record_date'].dt.day
    #data['week'] = data['record_date'].dt.isocalendar().week

In [26]:
for data in datasets:
    data['is_weekend'] = np.where(data['weekday'].isin(['Sunday','Saturday']),1,0)

In [27]:
for data in datasets:
    data['AVERAGE_DISTANCE_FREE'] = data['AVERAGE_FREE_FLOW_TIME'] * data['AVERAGE_FREE_FLOW_SPEED']
    data['AVERAGE_TIME'] = data['AVERAGE_TIME_DIFF'] + data['AVERAGE_FREE_FLOW_TIME'] 
    data['AVERAGE_SPEED'] =  data['AVERAGE_DISTANCE_FREE'] / data['AVERAGE_TIME']
    data['AVERAGE_SPEED_DIFF_Num'] =  data['AVERAGE_FREE_FLOW_SPEED'] - data['AVERAGE_SPEED']

In [28]:
corr = training.copy()

corr['AVERAGE_RAIN']=corr['AVERAGE_RAIN'].astype('category').cat.codes
corr['AVERAGE_RAIN'].sort_values(ascending=False)

corr['AVERAGE_CLOUDINESS']=corr['AVERAGE_CLOUDINESS'].astype('category').cat.codes
corr['AVERAGE_CLOUDINESS'].sort_values(ascending=False)

corr['AVERAGE_SPEED_DIFF']=corr['AVERAGE_SPEED_DIFF'].astype('category').cat.codes
corr['AVERAGE_SPEED_DIFF'].sort_values(ascending=False)

corr['LUMINOSITY']=corr['LUMINOSITY'].astype('category').cat.codes
corr['LUMINOSITY'].sort_values(ascending=False)

corr['weekday']=corr['weekday'].astype('category').cat.codes
corr['weekday'].sort_values(ascending=False)



873     6
768     6
4665    6
4671    6
773     6
       ..
5604    0
5603    0
2434    0
5599    0
2688    0
Name: weekday, Length: 6812, dtype: int8

In [29]:
#fig, ax = plt.subplots(figsize=(12,12))
#sns.heatmap(corr.corr(),cmap="PiYG")

In [30]:
training = training.drop(['city_name','record_date','AVERAGE_PRECIPITATION','AVERAGE_RAIN','AVERAGE_DISTANCE_FREE','AVERAGE_FREE_FLOW_SPEED'],axis=1)
testing = testing.drop(['city_name','record_date','AVERAGE_PRECIPITATION','AVERAGE_RAIN','AVERAGE_DISTANCE_FREE','AVERAGE_FREE_FLOW_SPEED'],axis=1)

In [31]:

training.loc[training.AVERAGE_CLOUDINESS == 'céu claro', 'AVERAGE_CLOUDINESS'] = 'céu limpo'
training.loc[training.AVERAGE_CLOUDINESS == 'nuvens quebrados', 'AVERAGE_CLOUDINESS'] = 'nuvens dispersas'
training.loc[training.AVERAGE_CLOUDINESS == 'nuvens quebradas', 'AVERAGE_CLOUDINESS'] = 'nuvens dispersas'
training.loc[training.AVERAGE_CLOUDINESS == 'algumas nuvens', 'AVERAGE_CLOUDINESS'] = 'nuvens dispersas'
training.loc[training.AVERAGE_CLOUDINESS == 'tempo nublado', 'AVERAGE_CLOUDINESS'] = 'nublado'


testing.loc[testing.AVERAGE_CLOUDINESS == 'céu claro', 'AVERAGE_CLOUDINESS'] = 'céu limpo'
testing.loc[testing.AVERAGE_CLOUDINESS == 'nuvens quebrados', 'AVERAGE_CLOUDINESS'] = 'nuvens dispersas'
testing.loc[testing.AVERAGE_CLOUDINESS == 'nuvens quebradas', 'AVERAGE_CLOUDINESS'] = 'nuvens dispersas'
testing.loc[testing.AVERAGE_CLOUDINESS == 'algumas nuvens', 'AVERAGE_CLOUDINESS'] = 'nuvens dispersas'
testing.loc[testing.AVERAGE_CLOUDINESS == 'tempo nublado', 'AVERAGE_CLOUDINESS'] = 'nublado'

In [32]:
testing['LUMINOSITY'].value_counts()

DARK         730
LIGHT        725
LOW_LIGHT     45
Name: LUMINOSITY, dtype: int64

In [33]:
training.loc[training.LUMINOSITY == 'LOW_LIGHT', 'LUMINOSITY'] = 'DARK'

testing.loc[testing.LUMINOSITY == 'LOW_LIGHT', 'LUMINOSITY'] = 'DARK'

In [34]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()


In [35]:
#profileTrain = ProfileReport(training)
#profileTest = ProfileReport(testing)

In [36]:
#profileTrain.to_notebook_iframe()
#profileTest.to_notebook_iframe()

In [37]:
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6812 entries, 0 to 6811
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   AVERAGE_SPEED_DIFF       6812 non-null   object 
 1   AVERAGE_TIME_DIFF        6812 non-null   float64
 2   AVERAGE_FREE_FLOW_TIME   6812 non-null   float64
 3   LUMINOSITY               6812 non-null   object 
 4   AVERAGE_TEMPERATURE      6812 non-null   float64
 5   AVERAGE_ATMOSP_PRESSURE  6812 non-null   float64
 6   AVERAGE_HUMIDITY         6812 non-null   float64
 7   AVERAGE_WIND_SPEED       6812 non-null   float64
 8   AVERAGE_CLOUDINESS       6812 non-null   object 
 9   weekday                  6812 non-null   int64  
 10  hour                     6812 non-null   int64  
 11  month                    6812 non-null   int64  
 12  year                     6812 non-null   int64  
 13  day                      6812 non-null   int64  
 14  is_weekend              

In [38]:

cyclical = CyclicalTransformer(variables=None, drop_original=True)

cic = cyclical.fit_transform(training[['weekday','hour','month','year','day']])
training['weekday_sin'] = cic['weekday_sin'].values
training['weekday_cos'] = cic['weekday_cos'].values

training['hour_sin'] = cic['hour_sin'].values
training['hour_cos'] = cic['hour_cos'].values

training['month_sin'] = cic['month_sin'].values
training['month_cos'] = cic['month_cos'].values

training['year_sin'] = cic['year_sin'].values
training['year_cos'] = cic['year_cos'].values

training['day_sin'] = cic['day_sin'].values
training['day_cos'] = cic['day_cos'].values

training = training.drop(['weekday','hour','month','year','day'],axis=1)

cic = cyclical.fit_transform(testing[['weekday','hour','month','year','day']])
testing['weekday_sin'] = cic['weekday_sin'].values
testing['weekday_cos'] = cic['weekday_cos'].values

testing['hour_sin'] = cic['hour_sin'].values
testing['hour_cos'] = cic['hour_cos'].values

testing['month_sin'] = cic['month_sin'].values
testing['month_cos'] = cic['month_cos'].values

testing['year_sin'] = cic['year_sin'].values
testing['year_cos'] = cic['year_cos'].values

testing['day_sin'] = cic['day_sin'].values
testing['day_cos'] = cic['day_cos'].values

testing = testing.drop(['weekday','hour','month','year','day'],axis=1)

In [39]:
training.head()

,AVERAGE_SPEED_DIFF,AVERAGE_TIME_DIFF,AVERAGE_FREE_FLOW_TIME,LUMINOSITY,AVERAGE_TEMPERATURE,AVERAGE_ATMOSP_PRESSURE,AVERAGE_HUMIDITY,AVERAGE_WIND_SPEED,AVERAGE_CLOUDINESS,is_weekend,...,weekday_sin,weekday_cos,hour_sin,hour_cos,month_sin,month_cos,year_sin,year_cos,day_sin,day_cos
0,Medium,11.5,71.4,LIGHT,15.0,1019.0,100.0,3.0,NoData,0,...,1.224647e-16,-1.0,0.942261,-0.334880,-8.660254e-01,-5.000000e-01,-2.449294e-16,1.000000,-0.394356,0.918958
1,High,48.3,87.4,LIGHT,21.0,1021.0,53.0,5.0,céu limpo,0,...,-8.660254e-01,-0.5,-0.631088,-0.775711,-8.660254e-01,-5.000000e-01,-3.112023e-03,0.999995,0.897805,-0.440394
2,High,38.4,85.2,LIGHT,26.0,1014.0,61.0,4.0,NoData,0,...,-2.449294e-16,1.0,-0.942261,-0.334880,-1.000000e+00,-1.836970e-16,-2.449294e-16,1.000000,0.201299,0.979530
3,High,61.0,94.1,LIGHT,18.0,1025.0,48.0,4.0,céu limpo,0,...,8.660254e-01,0.5,0.136167,-0.990686,8.660254e-01,5.000000e-01,-2.449294e-16,1.000000,-0.848644,0.528964
4,Medium,50.4,77.0,LIGHT,15.0,1008.0,82.0,10.0,NoData,0,...,1.224647e-16,-1.0,-0.136167,-0.990686,1.224647e-16,-1.000000e+00,-2.449294e-16,1.000000,0.937752,0.347305


In [40]:
for data in datasets:
    if 'AVERAGE_SPEED_DIFF' in data:
        data['AVERAGE_SPEED_DIFF'] = data['AVERAGE_SPEED_DIFF'].astype('category')
    data['AVERAGE_RAIN'] = data['AVERAGE_RAIN'].astype('category')
    data['LUMINOSITY'] = data['LUMINOSITY'].astype('category')
    data['AVERAGE_CLOUDINESS'] = data['AVERAGE_CLOUDINESS'].astype('category')

In [41]:
df_dummies_train = training[['AVERAGE_CLOUDINESS','LUMINOSITY']]

df_dummies_test = testing[['AVERAGE_CLOUDINESS','LUMINOSITY']]


In [42]:
for i in df_dummies_train:
    training = pd.concat([training,pd.get_dummies(training[i])],axis='columns').drop([i],axis='columns')

for i in df_dummies_test:
    testing = pd.concat([testing,pd.get_dummies(testing[i])],axis='columns').drop([i],axis='columns')

In [43]:
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6812 entries, 0 to 6811
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   AVERAGE_SPEED_DIFF       6812 non-null   object 
 1   AVERAGE_TIME_DIFF        6812 non-null   float64
 2   AVERAGE_FREE_FLOW_TIME   6812 non-null   float64
 3   AVERAGE_TEMPERATURE      6812 non-null   float64
 4   AVERAGE_ATMOSP_PRESSURE  6812 non-null   float64
 5   AVERAGE_HUMIDITY         6812 non-null   float64
 6   AVERAGE_WIND_SPEED       6812 non-null   float64
 7   is_weekend               6812 non-null   int64  
 8   AVERAGE_TIME             6812 non-null   float64
 9   AVERAGE_SPEED            6812 non-null   float64
 10  AVERAGE_SPEED_DIFF_Num   6812 non-null   float64
 11  weekday_sin              6812 non-null   float64
 12  weekday_cos              6812 non-null   float64
 13  hour_sin                 6812 non-null   float64
 14  hour_cos                

In [44]:
training.to_csv('processedTrain.csv',index=False)
testing.to_csv('processedTest.csv',index=False)